Majority vote from team's CV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob

from tqdm import tqdm
import seaborn as sns
from collections import Counter

import cv2

In [2]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [3]:
from collections import Counter
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [4]:
from collections import defaultdict

In [5]:
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [6]:
#exps = !ls ../input/whale-21-3-2022/ | grep pbz2 | grep fold4
#exps

In [7]:
FOLD=3

In [8]:
exps_all = [
    [
         'exp16v6Augmix_fold0.pbz2',
         'exp16v4sampler_fold0.pbz2',
         'exp16v6AugMixNew4_fold0.pbz2',
         'exp16v6AugMixNew4_fold0_tta.pbz2',
         'exp16v6AugmixB3_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0_remove_background.pbz2',
         'exp16v6AugmixB4_fold0.pbz2',
         'exp16v6AugmixB4_fold0_remove_background.pbz2',
         'exp16v6AugmixFullB4_fold0.pbz2',
         'exp16v6sampler_fold0.pbz2',
         'exp21b6_fold0.pbz2',
         'exp21b6_fold0_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold1.pbz2',
         'exp16v6AugMixNew4_fold1.pbz2',
         'exp16v6AugMixNew4_fold1_tta.pbz2',
         'exp16v6AugmixB3_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1_remove_background.pbz2',
         'exp16v6AugmixB4_fold1.pbz2',
         'exp21b6_fold1_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold2.pbz2',
         'exp16v6AugMixNew4_fold2.pbz2',
         'exp16v6AugMixNew4_fold2_tta.pbz2',
         'exp16v6AugmixB3_fold2.pbz2',
         'exp16v6AugmixB4Full_fold2.pbz2',
         'exp16v6AugmixB4Step2_fold2.pbz2',
         'exp16v6AugmixB4_fold2.pbz2',
         'exp16v6AugmixB4_fold2_remove_background.pbz2',
         'exp21b6_fold2_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold3.pbz2',
         'exp16v6AugMix1536_fold3.pbz2',
         'exp16v6AugmixB3_fold3.pbz2',
         'exp16v6AugmixB4Full_fold3.pbz2',
         'exp16v6AugmixB4Step2_fold3.pbz2',
         'exp16v6AugmixB4_fold3_remove_background.pbz2',
         'exp21b6_fold3_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold4.pbz2',
         'exp16v6AugMix1536_fold4.pbz2',
         'exp16v6AugmixB3_fold4.pbz2',
         'exp16v6AugmixB4Step2_fold4.pbz2',
         'exp16v6AugmixB4_fold4_remove_background.pbz2',
    ]
]

In [9]:
exps = exps_all[FOLD]

In [10]:
fold2 = pd.read_csv("/kaggle/happywhale/input/whale-21-3-2022/fold2.csv")
fold2.shape

(51033, 8)

In [11]:
valid = fold2.query("fold == @FOLD").reset_index()
train = fold2.query("fold != @FOLD").reset_index()

In [12]:
valid_ind = (fold2.fold == FOLD).values
train_ind = (fold2.fold != FOLD).values

In [13]:
train_inddividual_id_lis = train["individual_id"].values
valid_inddividual_id_lis = valid["individual_id"].values

def get_result_each(files):
    DIRNAME ="../input/whale-21-3-2022/"
      
    ret = []
        
    for fname in files:
        print(fname)
        data = decompress_pickle(DIRNAME + fname  )
        
        train_emb = data["emb_val"]#[new_index,:]
        
        train_emb=normalize(train_emb)
        
        valid_preds = train_emb[valid_ind]
        train_preds = train_emb[train_ind]
        
        
        distances = pairwise_distances(valid_preds,train_preds)
                
        K=1000
        predicted_positions = np.argpartition(distances, K, axis=1)[:, :K]

        all_pred_lis = []
        for i in tqdm(range(predicted_positions.shape[0])):
            nearest = [(train_inddividual_id_lis[j], distances[i, j]) for j in predicted_positions[i]]
            nearest.sort(key=lambda x: x[1])

            prediction = [index_id for index_id, d in nearest]

            done = set()
            pred_list = []
            for pred_id in prediction:
                if pred_id in done:
                    continue
                done.add(pred_id)
                pred_list.append(pred_id)
                if len(pred_list)==20:
                    break

            all_pred_lis.append(pred_list)
        
        ret.append(all_pred_lis)
        
    return ret
    

### get result for each model

In [14]:
ret = get_result_each(exps)

exp16v6Augmix_fold3.pbz2


100%|██████████| 8374/8374 [00:07<00:00, 1102.78it/s]


exp16v6AugMix1536_fold3.pbz2


100%|██████████| 8374/8374 [00:07<00:00, 1106.53it/s]


exp16v6AugmixB3_fold3.pbz2


...

100%|██████████| 8374/8374 [00:07<00:00, 1100.21it/s]


exp16v6AugmixB4_fold3_remove_background.pbz2


100%|██████████| 8374/8374 [00:07<00:00, 1104.83it/s]


exp21b6_fold3_remove_background.pbz2


100%|██████████| 8374/8374 [00:07<00:00, 1112.15it/s]


In [15]:
len(ret),len(ret[0]),len(ret[0][0])

(7, 8374, 20)

In [16]:
def get_score(weights):
    
    #train_emb
    vote_result = defaultdict(lambda:defaultdict(int))
    
    for preds, w in zip(ret, weights):
        for i in range(len(preds)):
            
            for j,p in enumerate(preds[i]):
                vote_result[i][p] += (w / (j+1))
        
        
    score_lis = []
    pred_ids = []

    for i in range(len(ret[0])):
        pred_list = np.array(sorted(vote_result[i].items(), key=lambda item: item[1])[::-1])[:5,0].tolist()

        if valid_inddividual_id_lis[i] in pred_list:
            s = 1/(pred_list.index(valid_inddividual_id_lis[i]) + 1)
            score_lis.append(s)
        else:
            score_lis.append(0)
    
    return np.mean(score_lis)

In [17]:
# un-weighted result
get_score([1] * len(exps))

0.8517215985988377

optimize by optuna

In [18]:
import optuna

In [19]:
def objective_wrapper(exps):
    def objective(trial):

        weights = [1]
        for i in range(len(exps)-1):
            w=trial.suggest_uniform(exps[i+1], 0, 4)
            weights.append(w)
        
        
        score_lis = get_score(weights)
        score = np.mean(score_lis)
        return score
    
    return objective


In [20]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective_wrapper(exps), timeout=300)

[I 2022-04-17 07:16:39,264] A new study created in memory with name: no-name-49b6dc4f-dfcb-4f20-b17a-1ea87aaaaece
[I 2022-04-17 07:16:40,482] Trial 0 finished with value: 0.85026271793647 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 0.5327470186830929, 'exp16v6AugmixB3_fold3.pbz2': 2.042939858252668, 'exp16v6AugmixB4Full_fold3.pbz2': 3.95165790889262, 'exp16v6AugmixB4Step2_fold3.pbz2': 0.6534111885286888, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 3.212802291750049, 'exp21b6_fold3_remove_background.pbz2': 1.6601877184809606}. Best is trial 0 with value: 0.85026271793647.
[I 2022-04-17 07:16:41,717] Trial 1 finished with value: 0.8559449884563332 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.676763132913788, 'exp16v6AugmixB3_fold3.pbz2': 3.2237758357971735, 'exp16v6AugmixB4Full_fold3.pbz2': 3.8782234563805957, 'exp16v6AugmixB4Step2_fold3.pbz2': 3.698927513226225, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 2.547103442164431, 'exp21b6_fold3_remove_background.pbz2': 1.53

[I 2022-04-17 07:17:01,336] Trial 17 finished with value: 0.8562514927155481 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.4097440892625492, 'exp16v6AugmixB3_fold3.pbz2': 0.6676799186494113, 'exp16v6AugmixB4Full_fold3.pbz2': 1.8941577645953567, 'exp16v6AugmixB4Step2_fold3.pbz2': 3.013134948861107, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 2.2253575754803245, 'exp21b6_fold3_remove_background.pbz2': 0.07235797661760196}. Best is trial 15 with value: 0.8635498766021815.
[I 2022-04-17 07:17:02,564] Trial 18 finished with value: 0.8554713000557281 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 1.7704331217560438, 'exp16v6AugmixB3_fold3.pbz2': 1.3974262914288613, 'exp16v6AugmixB4Full_fold3.pbz2': 3.6055950495465776, 'exp16v6AugmixB4Step2_fold3.pbz2': 3.2886363271915453, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.590562028901024, 'exp21b6_fold3_remove_background.pbz2': 1.131124684319512}. Best is trial 15 with value: 0.8635498766021815.
[I 2022-04-17 07:17:03,816] Trial 19

[I 2022-04-17 07:17:22,201] Trial 34 finished with value: 0.8624791019823262 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.050971100926793, 'exp16v6AugmixB3_fold3.pbz2': 0.4653898475771197, 'exp16v6AugmixB4Full_fold3.pbz2': 3.2341278527394715, 'exp16v6AugmixB4Step2_fold3.pbz2': 3.1688197951625603, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 0.8983701916130056, 'exp21b6_fold3_remove_background.pbz2': 0.44956806098291585}. Best is trial 21 with value: 0.8637210413183664.
[I 2022-04-17 07:17:23,426] Trial 35 finished with value: 0.8632234694689913 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.307783730436123, 'exp16v6AugmixB3_fold3.pbz2': 0.8613444950317238, 'exp16v6AugmixB4Full_fold3.pbz2': 3.227658742123908, 'exp16v6AugmixB4Step2_fold3.pbz2': 2.442150900256928, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.2612061488324207, 'exp21b6_fold3_remove_background.pbz2': 0.3523974033274263}. Best is trial 21 with value: 0.8637210413183664.
[I 2022-04-17 07:17:24,689] Trial 36 

[I 2022-04-17 07:17:43,089] Trial 51 finished with value: 0.8639738078178489 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.802071795707695, 'exp16v6AugmixB3_fold3.pbz2': 0.16199737879645937, 'exp16v6AugmixB4Full_fold3.pbz2': 3.5930182833673343, 'exp16v6AugmixB4Step2_fold3.pbz2': 2.4709575529444456, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.0160292924730434, 'exp21b6_fold3_remove_background.pbz2': 0.2186249870259366}. Best is trial 43 with value: 0.8641967200063688.
[I 2022-04-17 07:17:44,315] Trial 52 finished with value: 0.8500199028739751 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 0.008440993446316991, 'exp16v6AugmixB3_fold3.pbz2': 0.1544297215521046, 'exp16v6AugmixB4Full_fold3.pbz2': 3.643375650854924, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.975189697038746, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.105799027788579, 'exp21b6_fold3_remove_background.pbz2': 0.6548978556329144}. Best is trial 43 with value: 0.8641967200063688.
[I 2022-04-17 07:17:45,581] Trial 5

[I 2022-04-17 07:18:03,953] Trial 68 finished with value: 0.8624771116949288 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.995839416297361, 'exp16v6AugmixB3_fold3.pbz2': 1.115118261400751, 'exp16v6AugmixB4Full_fold3.pbz2': 3.337660186230847, 'exp16v6AugmixB4Step2_fold3.pbz2': 2.720840240503498, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.5274241433359055, 'exp21b6_fold3_remove_background.pbz2': 0.40036707142103173}. Best is trial 62 with value: 0.8644076904705039.
[I 2022-04-17 07:18:05,172] Trial 69 finished with value: 0.8634563330944988 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.238915493373219, 'exp16v6AugmixB3_fold3.pbz2': 0.6454097258029675, 'exp16v6AugmixB4Full_fold3.pbz2': 3.710642325498956, 'exp16v6AugmixB4Step2_fold3.pbz2': 2.3554977484909316, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.1485843166652974, 'exp21b6_fold3_remove_background.pbz2': 0.09057164900949763}. Best is trial 62 with value: 0.8644076904705039.
[I 2022-04-17 07:18:06,401] Trial 70 f

...

[I 2022-04-17 07:20:30,132] Trial 187 finished with value: 0.8640076427036064 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.106230532787811, 'exp16v6AugmixB3_fold3.pbz2': 0.34658373632842926, 'exp16v6AugmixB4Full_fold3.pbz2': 3.380833066523496, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.5179222996978603, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 2.335730927219096, 'exp21b6_fold3_remove_background.pbz2': 0.24084138014041115}. Best is trial 165 with value: 0.8649570097922139.
[I 2022-04-17 07:20:31,356] Trial 188 finished with value: 0.8647082238675264 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.4944834689497135, 'exp16v6AugmixB3_fold3.pbz2': 0.1458312623067566, 'exp16v6AugmixB4Full_fold3.pbz2': 3.544577045343423, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.6102326129070836, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 2.0361146746276324, 'exp21b6_fold3_remove_background.pbz2': 0.605493259778642}. Best is trial 165 with value: 0.8649570097922139.
[I 2022-04-17 07:20:32,580] Trial

[I 2022-04-17 07:20:50,990] Trial 204 finished with value: 0.8647858450760291 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.3487286945420927, 'exp16v6AugmixB3_fold3.pbz2': 0.3295822364677097, 'exp16v6AugmixB4Full_fold3.pbz2': 3.556580612192035, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.5068802724658854, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.7945103599934897, 'exp21b6_fold3_remove_background.pbz2': 0.5957685018985122}. Best is trial 165 with value: 0.8649570097922139.
[I 2022-04-17 07:20:52,214] Trial 205 finished with value: 0.8649430777804316 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.3442673773912612, 'exp16v6AugmixB3_fold3.pbz2': 0.360825623029255, 'exp16v6AugmixB4Full_fold3.pbz2': 3.5859115429819193, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.476633981953331, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.7914108199611294, 'exp21b6_fold3_remove_background.pbz2': 0.6541943330093867}. Best is trial 165 with value: 0.8649570097922139.
[I 2022-04-17 07:20:53,438] Trial

[I 2022-04-17 07:21:11,870] Trial 221 finished with value: 0.8646843404187565 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.3895232566009104, 'exp16v6AugmixB3_fold3.pbz2': 0.3080529873969067, 'exp16v6AugmixB4Full_fold3.pbz2': 3.517913083933236, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.5491527512155778, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.621370009416088, 'exp21b6_fold3_remove_background.pbz2': 0.5686069983556012}. Best is trial 207 with value: 0.8650147281267415.
[I 2022-04-17 07:21:13,097] Trial 222 finished with value: 0.8650465727251014 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.295290456092752, 'exp16v6AugmixB3_fold3.pbz2': 0.4093657585825101, 'exp16v6AugmixB4Full_fold3.pbz2': 3.5530387878095318, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.3865105824991601, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.7813136043136013, 'exp21b6_fold3_remove_background.pbz2': 0.663427189869461}. Best is trial 222 with value: 0.8650465727251014.
[I 2022-04-17 07:21:14,322] Trial 

[I 2022-04-17 07:21:32,757] Trial 238 finished with value: 0.8648435634105566 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.4323926118671286, 'exp16v6AugmixB3_fold3.pbz2': 0.3408992289283589, 'exp16v6AugmixB4Full_fold3.pbz2': 3.4627795765562412, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.1880600556971284, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.7992350232683092, 'exp21b6_fold3_remove_background.pbz2': 0.5748640137436476}. Best is trial 235 with value: 0.8651341453705914.
[I 2022-04-17 07:21:33,984] Trial 239 finished with value: 0.864403709895709 and parameters: {'exp16v6AugMix1536_fold3.pbz2': 3.238394405337151, 'exp16v6AugmixB3_fold3.pbz2': 0.5710941288191143, 'exp16v6AugmixB4Full_fold3.pbz2': 3.5731115652407084, 'exp16v6AugmixB4Step2_fold3.pbz2': 1.368918065319662, 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.9805250808266603, 'exp21b6_fold3_remove_background.pbz2': 0.5735697935216283}. Best is trial 235 with value: 0.8651341453705914.
[I 2022-04-17 07:21:35,209] Trial

CPU times: user 5min 1s, sys: 831 ms, total: 5min 1s
Wall time: 5min 1s


In [21]:
study.trials_dataframe().sort_values("value")

,number,value,datetime_start,datetime_complete,duration,params_exp16v6AugMix1536_fold3.pbz2,params_exp16v6AugmixB3_fold3.pbz2,params_exp16v6AugmixB4Full_fold3.pbz2,params_exp16v6AugmixB4Step2_fold3.pbz2,params_exp16v6AugmixB4_fold3_remove_background.pbz2,params_exp21b6_fold3_remove_background.pbz2,state
16,16,0.843973,2022-04-17 07:16:58.894567,2022-04-17 07:17:00.115997,0 days 00:00:01.221430,2.697381,0.503704,0.239388,3.953294,0.521451,1.016072,COMPLETE
9,9,0.844411,2022-04-17 07:16:50.339211,2022-04-17 07:16:51.549694,0 days 00:00:01.210483,2.017525,0.289534,0.918293,3.272684,2.964444,1.442136,COMPLETE
30,30,0.845701,2022-04-17 07:17:16.077154,2022-04-17 07:17:17.316063,0 days 00:00:01.238909,0.837373,0.708981,2.864605,2.626723,3.680165,0.942193,COMPLETE
50,50,0.847208,2022-04-17 07:17:40.646479,2022-04-17 07:17:41.868997,0 days 00:00:01.222518,1.193095,0.399644,3.109036,2.181470,3.986281,0.244227,COMPLETE
38,38,0.848233,2022-04-17 07:17:25.918507,2022-04-17 07:17:27.143196,0 days 00:00:01.224689,0.765446,2.188497,3.375747,3.490519,1.376580,1.631757,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
224,224,0.865023,2022-04-17 07:21:14.324021,2022-04-17 07:21:15.541378,0 days 00:00:01.217357,3.295908,0.465545,3.574465,1.206996,1.856750,0.693013,COMPLETE
241,241,0.865025,2022-04-17 07:21:35.210511,2022-04-17 07:21:36.440732,0 days 00:00:01.230221,3.434139,0.319566,3.494004,1.167880,1.859177,0.494563,COMPLETE
222,222,0.865047,2022-04-17 07:21:11.871971,2022-04-17 07:21:13.097472,0 days 00:00:01.225501,3.295290,0.409366,3.553039,1.386511,1.781314,0.663427,COMPLETE
234,234,0.865114,2022-04-17 07:21:26.575431,2022-04-17 07:21:27.802349,0 days 00:00:01.226918,3.410813,0.325359,3.581378,1.377333,1.788234,0.531877,COMPLETE


In [22]:
study.best_params

{'exp16v6AugMix1536_fold3.pbz2': 3.386816684889561,
 'exp16v6AugmixB3_fold3.pbz2': 0.47983335878908695,
 'exp16v6AugmixB4Full_fold3.pbz2': 3.585087261884701,
 'exp16v6AugmixB4Step2_fold3.pbz2': 1.3031034094501783,
 'exp16v6AugmixB4_fold3_remove_background.pbz2': 1.7710281641378367,
 'exp21b6_fold3_remove_background.pbz2': 0.5294771289417232}

In [23]:
best_weights = list(study.best_params.values())

In [24]:
# weighted result
get_score([1] + best_weights)

0.8651341453705914

In [25]:
# un-weighted result
get_score([1] * len(exps))

0.8517215985988377